In [1]:
import os
import pandas as pd
from openai import OpenAI
import re
# 寫入excel
from openpyxl import Workbook

d:\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [13]:
# 設定工作環境
work_folders =['【正式】202405'] 
names =  ['國泰','玉山','台北富邦', '兆豐', '中信', '台新']
input_folder = '003_第二階段摘要'
output_folder = '【正式】分析'
file_suffix_name = 'PreliminaryAnalysis'
only_qualitative_ = True

# 函式

In [9]:
# 資料導入
df = pd.read_excel(r"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\【正式】all to one\銀行量化.xlsx")
df2 = pd.read_excel(r"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\【正式】all to one\銀行逾放比及備呆覆蓋率(單位百萬).xlsx")

'''準備資料'''
def make_the_file_for_GPT_(path, work_folder, file_list,name, df, df2):
    ym = work_folder[-6:-2]+'_'+ work_folder[-2:]
    final_abstract = ''
    for i in file_list:
        if i[:2]==name[:2]:
            with open(path+'\\'+i, "r") as file:
                final_abstract += '\n'+file.read()
    if name[:2] == '台北':
        name = '富邦'
    pick = (df['月份']==ym) & (df['銀行']==name)
    df_m = df[pick]
    column_names = ['銀行', '類別', '月份', '時間', '本月', '前期', '月變化', '當月排名', '數字_名次變動', '數字_名次更動狀況']
    df_m = df_m.loc[:, column_names]
    pick2 = (df2['月份']==ym) & (df2['銀行']==name)
    df_L = df2[pick2]
    return final_abstract, df_m, df_L

'''量化質性綜合分析'''
def QQ_gpt_analyzing(name, final_abstract, df_m, df_L):
    client = OpenAI(api_key = "")
    # 互動
    print(name, final_abstract, df_m)
    completion = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": f"你是一位使用zhtw為主要語言的專業投顧分析師，擅長領域是銀行業。職責在於對{name}銀行的新聞摘要(質性)、財務與信用卡數據(量化)進行質性與量化結合的的綜合分析"},
                    {"role": "user", "content": f'''背景:一份新聞摘要文本、一份銀行財務與信用卡數據。
                    任務1(綜合分析):請仔細閱讀質性與量化兩份資料後，產出一份結合數據與新聞的分析，向你的客戶說明{name}銀行的市場優勢與劣勢、未來發展機會、風險(包含但不限於營運、舞弊)及其他應注意事項。
                    任務2(量化分析(收入面)):請以我提供給妳的質性資料為背景，針對"月營收"、"利息收入"、"手續費收入"、"信用卡有效卡數"來進行深入的文字分析與說明
                    任務3(量化分析(放款面)):請以我提供給妳的質性資料為背景，針對"逾放比"、"備抵呆帳/逾期放款(%)"來進行深入的文字分析與說明
                    任務4(量化數字變動原因):請以我提供給妳的質性資料為背景，來解釋量化資料中數字的變動原因。
                    
                    量化資料(收入面):{df_m} 
                    量化資料(放款面):{df_L}
                    質性資料:{final_abstract}
                    
                    注意1:請將字數限制在1500字以內。
                    注意2:請務必依照格式回答，小標需縮排。
                    注意3:若有你無法分析的項目，請直接於該小標內填入"無法回答"，切勿虛構！

                    指定回傳格式:
                    ### 綜合分析
                    
                    **市場優勢**：
                        1. xx
                        2. xx
                        3. xx...

                    **市場劣勢**：
                        1. xx
                        2. xx
                        3. xx...

                    **發展機會**：
                        1. xx
                        2. xx
                        3. xx...

                    **風險**：
                        1. xx
                        2. xx
                        3. xx...

                    **應注意事項**：
                        1. xx
                        2. xx
                        3. xx...

                    **其他**：
                        1. xx
                        2. xx
                        3. xx...
                    
                    ### 量化分析
                        1. xx
                        2. xx
                        3. xx
                        4. xx...

                    ### 量化數字變動原因
                        1. xx
                        2. xx
                        3. xx
                        4. xx...
                        '''}
                ],
                temperature=0.95,
                )
    response = completion.choices[0].message.content
    return response

'''僅有質性分析'''
def onlyone_Q_gpt_analyzing(name, final_abstract):
    client_2 = OpenAI(api_key = "")
    # 互動
    completion_2 = client_2.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": f"你是一位使用zhtw為主要語言的專業投顧分析師，擅長領域是銀行業。職責在於對{name}銀行的新聞摘要進行深度分析"},
                    {"role": "user", "content": f'''背景:一份新聞摘要文本
                    任務1(綜合分析):請仔細閱讀新聞質性資料後，產出一份結合你的所知與新聞的分析，向你的客戶說明{name}銀行的市場優勢與劣勢、未來發展機會、風險(包含但不限於營運、舞弊)及其他應注意事項。
                    
                    質性資料:{final_abstract}
                    
                    注意1:請將字數限制在1500字以內。
                    注意2:請務必依照格式回答，小標需縮排。
                    注意3:若有你無法分析的項目，請直接於該小標內填入"無法回答"，切勿虛構！

                    指定回傳格式:
                    ### 綜合分析
                    
                    **市場優勢**：
                        1. xx
                        2. xx
                        3. xx...

                    **市場劣勢**：
                        1. xx
                        2. xx
                        3. xx...

                    **發展機會**：
                        1. xx
                        2. xx
                        3. xx...

                    **風險**：
                        1. xx
                        2. xx
                        3. xx...

                    **應注意事項**：
                        1. xx
                        2. xx
                        3. xx...

                    **其他**：
                        1. xx
                        2. xx
                        3. xx...    
                        '''}
                ],
                temperature=0.95,
                )
    response_2 = completion_2.choices[0].message.content
    return response_2

# 開始

In [ ]:
if only_qualitative_ == False:
    print("QQ")
    for work_folder in work_folders[1:]: 
        path = rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{input_folder}\DOCX'
        file_list = os.listdir(path)
        for name in names:
            final_abstract, df_m, df_L = make_the_file_for_GPT_(path, work_folder, file_list,name, df, df2)
            if len(final_abstract)==0 or len(df_m)==0 or len(df_L)==0:
                print('no data')
                break
            print(f'{work_folder[4:]} {name} is processing')
            response = QQ_gpt_analyzing(name, final_abstract, df_m, df_L)
            with open(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{output_folder}\{work_folder[4:]}_{name}_{file_suffix_name}.docx", "w") as file:
                for i in response:
                    file.write(i)

else:
    print('Q')
    for work_folder in work_folders: 
        path = rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{input_folder}'
        file_list = os.listdir(path)
        for name in names:
            final_abstract, df_m, df_L = make_the_file_for_GPT_(path, work_folder, file_list,name, df, df2)
            if len(final_abstract)==0:
                print('no data')
                break
            print(f'{work_folder[4:]} {name} is processing')
            response = onlyone_Q_gpt_analyzing(name, final_abstract)
            with open(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{output_folder}\{work_folder[4:]}_{name}_{file_suffix_name}.docx", "w") as file:
                for i in response:
                    file.write(i)

# 合併

In [19]:
# 設定工作環境
input_folder ='【正式】分析'
output_folder = '【正式】all to one'
file_name = 'GPT分析'

In [20]:
file_list = os.listdir(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{input_folder}")

df = pd.DataFrame()

for xx in file_list:
    print(xx)
    with open(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{input_folder}\{xx}", "r") as file:
        preliminary_analysis = file.read()
    each_sentences = re.findall(r"(.*?)[\n。$]", preliminary_analysis, re.MULTILINE) 

    if xx.split('_')[1] == '台北富邦':
        name = '富邦'
    else:
        name = xx.split('_')[1]
    temp_df = pd.DataFrame({'分析內文':each_sentences,
                            '年月':[xx.split('_')[0]]*len(each_sentences),
                            '銀行':[name]*len(each_sentences)})
    df = pd.concat([temp_df, df], axis=0)

df.to_excel(rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{output_folder}\{file_name}.xlsx', index_label='index')

202401_中信_PreliminaryAnalysis.docx
202401_兆豐_PreliminaryAnalysis.docx
202401_台北富邦_PreliminaryAnalysis.docx
202401_台新_PreliminaryAnalysis.docx
202401_國泰_PreliminaryAnalysis.docx
202401_玉山_PreliminaryAnalysis.docx
202402_中信_PreliminaryAnalysis.docx
202402_兆豐_PreliminaryAnalysis.docx
202402_台北富邦_PreliminaryAnalysis.docx
202402_台新_PreliminaryAnalysis.docx
202402_國泰_PreliminaryAnalysis.docx
202402_玉山_PreliminaryAnalysis.docx
202403_中信_PreliminaryAnalysis.docx
202403_兆豐_PreliminaryAnalysis.docx
202403_台北富邦_PreliminaryAnalysis.docx
202403_台新_PreliminaryAnalysis.docx
202403_國泰_PreliminaryAnalysis.docx
202403_玉山_PreliminaryAnalysis.docx
202404_中信_PreliminaryAnalysis.docx
202404_兆豐_PreliminaryAnalysis.docx
202404_台北富邦_PreliminaryAnalysis.docx
202404_台新_PreliminaryAnalysis.docx
202404_國泰_PreliminaryAnalysis.docx
202404_玉山_PreliminaryAnalysis.docx
202405_中信_PreliminaryAnalysis.docx
202405_兆豐_PreliminaryAnalysis.docx
202405_台北富邦_PreliminaryAnalysis.docx
202405_台新_PreliminaryAnalysis.docx
202405_國泰_

<ipython-input-20-c56c4978c8b1>:20: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  df.to_excel(rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{output_folder}\{file_name}.xlsx', index_label='index')
